# Introduction/Business Problem

Given a specific borough of a city chosen, namely Rabat, Morocco's Hay Riad. Can we explain the prices of renting an appartment by a similarity with prestigious areas of Paris, France.
If so, then everything would make sense. To do so, we need to compare and cluster Paris's arrondissments with Hay Riad included so we get to know which areas of Paris are similar to Hay Riad, based on the Foursquare dataset.

First things first, we need to import the libraries to work with

In [58]:
import pandas as pd
import numpy as np
import requests
import json

import folium

In [4]:
paris_geojson = requests.get("https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/paris.geojson").json()

We need to turn `.geojson` file into pandas' `DataFrame` object.

In [64]:
dfParis = pd.DataFrame(columns=['Postal Code', 'Borough', 'Latitude', 'Longitude'])

import statistics

for feature in paris_geojson['features']:
    #print(json.dumps(feature,indent=2))
    code = feature['properties']['cartodb_id']
    borough = feature['properties']['name']
    equilibrium = feature["geometry"]["coordinates"][0][0]
    #print(json.dumps(equilibrium, indent=2))
    def select(obj, idx):
        for o in obj:
            yield o[idx]
    longitude = statistics.mean(list(select(equilibrium,0)))
    latitude = statistics.mean(list(select(equilibrium,1)))
    
    value = {'Postal Code':75000 + code, 'Borough': borough, 'Latitude': latitude, 'Longitude': longitude}
    print(json.dumps(value, indent = 2))
    dfParis = dfParis.append(value, ignore_index=True)

dfParis = dfParis.append({'Postal Code':10000, 'Borough': 'Hay Riad', 'Latitude':33.958752, 'Longitude':-6.871090}, ignore_index=True)

{
  "Postal Code": 75002,
  "Borough": "Bourse",
  "Latitude": 48.869408125,
  "Longitude": 2.3399715
}
{
  "Postal Code": 75003,
  "Borough": "Temple",
  "Latitude": 48.86219675,
  "Longitude": 2.3603217499999998
}
{
  "Postal Code": 75005,
  "Borough": "Panth\u00e9on",
  "Latitude": 48.84591771428572,
  "Longitude": 2.3516000714285714
}
{
  "Postal Code": 75006,
  "Borough": "Luxembourg",
  "Latitude": 48.85151675,
  "Longitude": 2.3316077500000003
}
{
  "Postal Code": 75007,
  "Borough": "Palais-Bourbon",
  "Latitude": 48.853572176470585,
  "Longitude": 2.3168273529411763
}
{
  "Postal Code": 75008,
  "Borough": "\u00c9lys\u00e9e",
  "Latitude": 48.87250172413793,
  "Longitude": 2.3101774827586206
}
{
  "Postal Code": 75009,
  "Borough": "Op\u00e9ra",
  "Latitude": 48.87732752380953,
  "Longitude": 2.3366013333333333
}
{
  "Postal Code": 75010,
  "Borough": "Enclos-St-Laurent",
  "Latitude": 48.87698195454546,
  "Longitude": 2.3612876363636364
}
{
  "Postal Code": 75011,
  "Borough"

## Achieved so far...
What we achieved so far is to include a position for each arrondissment of Paris, they were given as the barycenters from a geojson served polygons. Since the vicinity is small, the barycenter as mean of the values of latitudes and longitudes independently is Ok... We can see it in the map, further below. For the map, even Rabat's Hay Riad is available with its appropriate Postal Code

In [65]:
dfParis

,Postal Code,Borough,Latitude,Longitude
0,75002,Bourse,48.869408,2.339971
1,75003,Temple,48.862197,2.360322
2,75005,Panthéon,48.845918,2.351600
3,75006,Luxembourg,48.851517,2.331608
4,75007,Palais-Bourbon,48.853572,2.316827
5,75008,Élysée,48.872502,2.310177
6,75009,Opéra,48.877328,2.336601
7,75010,Enclos-St-Laurent,48.876982,2.361288
8,75011,Popincourt,48.858493,2.381793
9,75013,Gobelins,48.827353,2.353338


In [66]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfParis['Latitude'], dfParis['Longitude'], dfParis['Borough'], dfParis['Postal Code']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris